In [174]:
from urllib import request
from bs4 import BeautifulSoup
import re
import jieba
from collections import defaultdict
import pymysql

class crawler:
    def __init__(self, dbname):
        self.word_url = defaultdict(set)
        self.connect = pymysql.connect(host="localhost", user="root", password="root", charset="utf8mb4", db="searchengine")
        self.cursor = self.connect.cursor()
        
    def __del__(self):
        pass
    
    def dbcommit(self):
        self.connect.commit()
    
    def addtoindex(self,url,soup):
#         if not self.isindexed(url):
#             print('xxxx')
            
        urlid = self.getentryid('urllist','url',url)
        
        text = self.gettextonly(soup)
        words = self.separatewords(text)
        for i,word in enumerate(words):
            # self.word_url[word].add(url)
            wordid = self.getentryid('wordlist','word',word)
            self.cursor.execute('insert into wordlocation(urlid,wordid,location) values (%s,%s,%s)', (urlid, wordid, i))
    
    def getentryid(self, table, field, value, createnew=True):
        self.cursor.execute("select id from %s" % table +" where %s" % field + " = %s", (value))
        res = self.cursor.fetchone()
        if res == None:
            self.cursor.execute("insert into %s" % table +" (%s " % field +") values (%s)", (value))
            self.dbcommit()
            return self.cursor.lastrowid
        else:
            return res[0]
    
    def gettextonly(self, soup):
        # 直接用下面代码处理有问题
        if soup.body:
            return soup.body.get_text().strip()
        else:
            return soup.get_text().strip()
        '''
        v = soup.string
        if v == None:
            c = soup.contents
            resulttext = ''
            for t in c:
                subtext = self.gettextonly(t)
                resulttext += subtext + '\n'
            return resulttext
        else:
            return v.strip()
        return v
        '''
    
    def separatewords(self,words):
        pattern = r'[a-zA-Z0-9\’!\"#$%&\'()*+,（,）,-\.\/\:;<=>?@，。?★、…【】《》？“”‘’！\[\]^_`{|}~\\\s]+'
        return [word.strip() for word in jieba.lcut(words) if not re.search(pattern, str(word))]
    
    def isindexed(self, url):
#         self.cursor.execute('select * from urllist where url = %s',url)
#         find = self.cursor.fetchone()
#         if find != None:
#             return find[0]
        return False
    def addlinkref(self, urlFrom, urlTo, linkText):
        pass
#         print(urlFrom, urlTo, linkText)
    def crawl(self, pages, depth=4):
        for i in range(depth):
            newpages = set()
            for page in pages:
                try:
                    c = request.urlopen(page)
                except:
                    print("Can't open %s" % page)
                    continue
                # 获取当前页面的html内容
                soup = BeautifulSoup(c.read(),'html')
                self.addtoindex(page, soup)
                
                links = soup('a')
                for link in links:
                    if 'href' in dict(link.attrs):
                        url = request.urljoin(page, link['href'])
                        newpages.add(url)
                        linkText = self.gettextonly(link)
                        self.addlinkref(page, url, linkText)
                    
            pages = newpages
            
    def createindextables(self):
        pass

In [175]:
pagelist = ['http://m.judarhr.com']
crawler = crawler('')
crawler.crawl(pagelist)

########################################################################################################################
########################################################################################################################
########################################################################################################################
Can't open http://www.beian.miit.gov.cn/
Can't open javascript:void(0);
Can't open javascript:;
########################################################################################################################
Can't open http://www.beian.miit.gov.cn/
Can't open javascript:void(0)


In [201]:
from collections import defaultdict
class searcher:
    def __init__(self):
        self.connect = pymysql.connect(host="localhost", user="root", password="root", charset="utf8mb4", db="searchengine")
        self.cursor = self.connect.cursor()
        
    def __del__(self):
        self.connect.close()
        
    def show(self,q):
        words = jieba.lcut(q)
        result = set()
        for word in words:
            self.cursor.execute('select id from wordlist where word = %s',(word))
            wordid = self.cursor.fetchone()[0]
            self.cursor.execute('select urlid from wordlocation where wordid = %s',(wordid))
            urlids = set(self.cursor.fetchall())
            for urlid in urlids:
                result.add(urlid[0])
        
        for id in result:
            self.cursor.execute('select * from urllist where id = %s',(id))
            url = self.cursor.fetchone()
            print(url)
        
s = searcher()
s.show('在线')

(1, 'http://m.judarhr.com')
(42, 'http://www.judarhr.com/index/index/articledetail.html?id=238')
(11, 'http://www.judarhr.com/index/index/articledetail.html?id=235')
